In [ ]:
# SORU 1
SELECT 
	CITY AS [ŞEHİR],
	SUM(LINETOTAL) AS [TOPLAM SATIŞ TUTARI]
FROM SALEORDERS
GROUP BY CITY
ORDER BY 1

In [ ]:
# SORU 2
SELECT 
	CITY AS [ŞEHİR],
	MONTH_ AS [AY],
	SUM(LINETOTAL) AS [TOPLAM SATIŞ TUTARI]
FROM SALEORDERS
GROUP BY
	CITY,
	MONTH_
ORDER BY 
	1, 2

In [ ]:
# SORU 3
SELECT
	CITY,
	DAYOFWEEK_,
	SUM(LINETOTAL) AS [EN YÜKSEK SATIŞ TUTARI]
FROM SALEORDERS
GROUP BY
	CITY,
	DAYOFWEEK_
ORDER BY
	1, 3 DESC

In [ ]:
# SORU 4
SELECT
	CITY,
	(SELECT SUM(LINETOTAL) FROM SALEORDERS WHERE DAYOFWEEK_='01.PZT' AND SO.CITY=CITY) AS [PAZARTESİ],
	(SELECT SUM(LINETOTAL) FROM SALEORDERS WHERE DAYOFWEEK_='02.SAL' AND SO.CITY=CITY) AS [SALI],
	(SELECT SUM(LINETOTAL) FROM SALEORDERS WHERE DAYOFWEEK_='03.ÇAR' AND SO.CITY=CITY) AS [ÇARŞAMBA],
	(SELECT SUM(LINETOTAL) FROM SALEORDERS WHERE DAYOFWEEK_='04.PER' AND SO.CITY=CITY) AS [PERŞEMBE],
	(SELECT SUM(LINETOTAL) FROM SALEORDERS WHERE DAYOFWEEK_='05.CUM' AND SO.CITY=CITY) AS [CUMA],
	(SELECT SUM(LINETOTAL) FROM SALEORDERS WHERE DAYOFWEEK_='06.CMT' AND SO.CITY=CITY) AS [CUMARTESİ],
	(SELECT SUM(LINETOTAL) FROM SALEORDERS WHERE DAYOFWEEK_='07.PAZ' AND SO.CITY=CITY) AS [PAZAR]
FROM SALEORDERS AS SO
GROUP BY CITY
ORDER BY CITY

In [ ]:
# SORU 5
# OPTION 1
SELECT 
	CITY,
	CATEGORY1,
	[TOPLAM SATIS]
FROM (
SELECT
	CITY,
	CATEGORY1,
	SUM(LINETOTAL) AS [TOPLAM SATIS],
	ROW_NUMBER() OVER (PARTITION BY CITY ORDER BY SUM(LINETOTAL) DESC) AS RANK_
FROM SALEORDERS
GROUP BY 
	CITY, 
	CATEGORY1) AS T
WHERE RANK_ < 6
ORDER BY 
	1,
	3 DESC
    
    
    # OPTION 2
    SELECT
	SO.CITY,
	SO2.CATEGORY1,
	MAX(SO2.TOPLAMSATIS) AS [SON TOPLAM]
FROM SALEORDERS AS SO
CROSS APPLY (
	SELECT TOP 5
		CATEGORY1, 
		SUM(LINETOTAL) AS TOPLAMSATIS 
	FROM SALEORDERS 
	WHERE CITY=SO.CITY 
	GROUP BY CATEGORY1 
	ORDER BY 2 DESC) AS SO2
GROUP BY
	SO.CITY,
	SO2.CATEGORY1
ORDER BY 
	1,
	3 DESC

In [ ]:
# SORU 6
SELECT 
	CITY,
	CATEGORY1,
	CATEGORY2,
	[TOPLAM SATIS],
	RANK_
FROM (
SELECT
	CITY,
	CATEGORY1,
	CATEGORY2,
	SUM(LINETOTAL) AS [TOPLAM SATIS],
	ROW_NUMBER() OVER (PARTITION BY CITY ORDER BY SUM(LINETOTAL) DESC) AS RANK_
FROM SALEORDERS
GROUP BY 
	CITY,
	CATEGORY1, 
	CATEGORY2) AS T
WHERE RANK_ < 4
ORDER BY 
	1,
	4 DESC

In [ ]:
# SORU 7
# OPTION 1
CREATE TABLE CITIES2(
	ID INT IDENTITY(1, 1) PRIMARY KEY,
	COUNTRYID INT,
	CITY VARCHAR(50)
)


INSERT INTO CITIES2 (COUNTRYID, CITY)
SELECT
	COUNTRYID,
	CITY
FROM CITIES


# OPTION 2
# PRIMARY KEY OZELLIGINI AKTARMAZ
SELECT * INTO CITIES2 FROM CITIES

In [ ]:
# SORU 7
SELECT 
	-- USERS
	US.USERNAME_, 
	US.NAMESURNAME,
	US.TELNR1,
	US.TELNR2,
	-- COUNTRIES
	CO.COUNTRY,
	-- CITIES
	CI.CITY,
	-- TOWNS
	[TO].TOWN,
	-- ADDRESS
	AD.ADDRESSTEXT,
	-- ORDERS
	[OR].ID AS [ORDERID],
	-- ITEMS
	IT.ITEMCODE,
	IT.ITEMNAME,
	IT.BRAND,
	IT.CATEGORY1,
	IT.CATEGORY2,
	IT.CATEGORY3,
	IT.CATEGORY4,
	-- ORDERDETAILS
	OD.AMOUNT,
	OD.UNITPRICE,
	OD.LINETOTAL,
	-- ORDERS
	CONVERT(DATE, [OR].DATE_) AS [ORDERDATE],
	CONVERT(TIME, [OR].DATE_) AS [ORDERTIME],
	YEAR([OR].DATE_) AS [YEAR_],
	DATENAME(MONTH, [OR].DATE_) AS [MONTH_],
	DATENAME(DW, [OR].DATE_) AS [DAYOFWEEK_]
INTO SALEORDERS2
FROM ORDERS AS [OR]
JOIN USERS AS [US] ON US.ID = [OR].USERID
JOIN ADDRESS AS [AD] ON AD.ID = [OR].ADDRESSID
JOIN COUNTRIES AS [CO] ON CO.ID = AD.COUNTRYID
JOIN CITIES AS [CI] ON CI.ID = AD.CITYID
JOIN TOWNS AS [TO] ON [TO].ID = AD.TOWNID
JOIN ORDERDETAILS AS [OD] ON OD.ORDERID = [OR].ID
JOIN ITEMS AS [IT] ON IT.ID = OD.ITEMID


# SALEORDERS TABLOSUNUN İÇERDİĞİ KAYITLARI DİĞER TABLOLARI INNER JOIN İLE BİRLEŞTİREREK ELDE ETTİK.
# SELECT column_name INTO new_table_name FROM table_name YAPISI İLE BU KAYITLARI YENİ BİR TABLOYA AKTARDIK.

In [ ]:
# SORU 8
SELECT 
	[CI].CITY AS [ŞEHİRLER],
	SUM([OR].TOTALPRICE) AS [TOPLAM SATIŞ]
FROM ORDERS AS [OR]
JOIN ADDRESS AS [AD] ON [AD].ID = [OR].ADDRESSID
JOIN CITIES AS [CI] ON [CI].ID = [AD].CITYID
GROUP BY [CI].CITY
ORDER BY 1


In [ ]:
# SORU 9
SELECT
	BRAND, 
	CATEGORY1, 
	TOTAL
FROM (
	SELECT
		BRAND,
		CATEGORY1,
		SUM(LINETOTAL) AS [TOTAL],
		ROW_NUMBER() OVER (PARTITION BY BRAND ORDER BY SUM(LINETOTAL) DESC, CATEGORY1 ) AS [RANK]
	FROM ITEMS AS [IT]
	JOIN ORDERDETAILS AS [OD] ON [OD].ITEMID = [IT].ID
	GROUP BY
		BRAND,
		CATEGORY1
	) AS [TEMPTABLE]
WHERE [RANK] = 1
ORDER BY 
	3 DESC

In [ ]:
# SORU 10
SELECT
	CATEGORY1,
	BRAND,
	TOTAL
FROM (
	SELECT
		BRAND,
		CATEGORY1,
		SUM(LINETOTAL) AS [TOTAL],
		ROW_NUMBER() OVER (PARTITION BY CATEGORY1 ORDER BY SUM(LINETOTAL) DESC, BRAND ) AS [RANK]
	FROM ITEMS AS [IT]
	JOIN ORDERDETAILS AS [OD] ON [OD].ITEMID = [IT].ID
	GROUP BY
		BRAND,
		CATEGORY1
	) AS [TEMPTABLE]
WHERE [RANK] = 1
ORDER BY 
	3 DESC

In [ ]:
# SORU 11
SELECT 
	[IT].ID,
	[IT].BRAND,
	[IT].ITEMNAME,
	[IT].CATEGORY1,
	MIN([OD].UNITPRICE) AS [MINPRICE],
	MAX([OD].UNITPRICE) AS [MAXPRICE],
	COUNT(DISTINCT [OD].UNITPRICE) AS [PRICECHANGES],
	AVG([OD].UNITPRICE) AS [AVGPRICE]
FROM ORDERDETAILS AS [OD]
JOIN ITEMS AS [IT] ON  [IT].ID = [OD].ITEMID
GROUP BY 
	[IT].ID,
	[IT].ITEMNAME,
	[IT].BRAND,
	[IT].CATEGORY1
ORDER BY 
	2, 
	4

In [ ]:
# SORU 12
SELECT 
	NAMESURNAME,
	(SELECT COUNT(DISTINCT ID) FROM ADDRESS WHERE USERID = USERS.ID GROUP BY USERID) AS [ADDRESSCOUNT],
	LASTORDER,
	ADDRESSID,
	ADDRESSTEXT	AS [LASTADDRESSTEXT]
FROM (
	SELECT
		ORDERS.USERID AS [USERID],
		ADDRESSID,
		MAX(DATE_) AS [LASTORDER],
		ROW_NUMBER() OVER (PARTITION BY ORDERS.USERID ORDER BY MAX(DATE_) DESC) AS [RANK]
	FROM ORDERS 
	JOIN ADDRESS ON ADDRESS.ID = ADDRESSID
	GROUP BY 
		ORDERS.USERID,
		ADDRESSID
) AS [TEMPTABLE]
JOIN USERS ON USERS.ID = USERID
JOIN ADDRESS ON ADDRESS.ID = ADDRESSID
WHERE [RANK] = 1
ORDER BY 1

In [ ]:
# SORU 13
SELECT 
	NAMESURNAME,
	(SELECT COUNT(DISTINCT ID) FROM ADDRESS WHERE USERID = USERS.ID GROUP BY USERID) AS [ADDRESSCOUNT],
	LASTORDER,
	ADDRESSID,
	ADDRESSTEXT	AS [LASTADDRESSTEXT],
	DISTRICT,
	TOWN,
	CITY
FROM (
	SELECT
		ORDERS.USERID AS [USERID],
		ADDRESSID,
		MAX(DATE_) AS [LASTORDER],
		ROW_NUMBER() OVER (PARTITION BY ORDERS.USERID ORDER BY MAX(DATE_) DESC) AS [RANK]
	FROM ORDERS 
	JOIN ADDRESS ON ADDRESS.ID = ADDRESSID
	GROUP BY 
		ORDERS.USERID,
		ADDRESSID
) AS [TEMPTABLE]
JOIN USERS ON USERS.ID = USERID
JOIN ADDRESS ON ADDRESS.ID = ADDRESSID
JOIN CITIES ON CITIES.ID = ADDRESS.CITYID
JOIN DISTRICTS ON DISTRICTS.ID = ADDRESS.DISTRICTID
JOIN TOWNS ON TOWNS.ID = ADDRESS.TOWNID
WHERE [RANK] = 1
ORDER BY 1

In [ ]:
# SORU 14
SELECT 
	CITY,
	[YEAR],
	COUNT([DAY]) AS [MONTHCOUNT]
FROM (
	SELECT 
		CITY,
		YEAR(DATE_) AS [YEAR],
		UPPER(DATENAME(MONTH, DATE_)) AS [MONTH],
		DAY(DATE_) AS [DAY],
		SUM(TOTALPRICE) AS [TOTALPRICE]
	FROM ORDERS AS [OR]
	JOIN ADDRESS AS [AD] ON [AD].ID = [OR].ADDRESSID
	JOIN CITIES AS [CI] ON [CI].ID = [AD].CITYID
	WHERE UPPER(DATENAME(MONTH, DATE_)) = 'JANUARY'
	GROUP BY
		CITY,
		YEAR(DATE_),
		UPPER(DATENAME(MONTH, DATE_)),
		DAY(DATE_)
	HAVING SUM(TOTALPRICE) < 500
) AS [TEMPTABLE]
GROUP BY 
	CITY,
	[YEAR]
HAVING COUNT([DAY]) > 10